In [1]:
import numpy as np    
import matplotlib.pyplot as plt       
from torchvision.transforms import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F

# Google Colab
from fastai.vision.all import *
set_seed(42, reproducible= True)
source = untar_data(URLs.IMAGENETTE)

In [2]:
classes = ("Tench", "English Springer", "Cassette Player", "Chain Saw", "Church", "French Horn", "Garbage Truck", "Gas Pump", "Golf Ball", "Parachute")
batch_size = 64
width = 224
# mean = [0.4655, 0.4546, 0.4251]
# std = [0.2775, 0.2725, 0.2938]
mean = [0.5,0.5,0.5]
std = [0.27,0.27,0.27]


def load_data():
  train = source/"train"
  val = source/"val"

  train_dataset = ImageFolder(
    train,
    transforms.Compose([
        transforms.Resize(width),
        transforms.RandomCrop(width), 
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
        transforms.RandomErasing()
    ]))

  val_dataset = ImageFolder(
    val,
    transforms.Compose([
        transforms.Resize(width),
        transforms.RandomCrop(width), 
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]))

  train_dataloader = DataLoader(train_dataset, batch_size, shuffle=True)
  val_dataloader = DataLoader(val_dataset, batch_size)
  
  return train_dataloader, val_dataloader

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Model

In [4]:
def std_conv(in_channels, out_channels, kernel, stride):
  return nn.Sequential(
      nn.Conv2d(in_channels, out_channels, kernel_size=kernel, stride=stride, padding=1, bias=False),
      nn.BatchNorm2d(out_channels)
  )

class block(nn.Module):
  def __init__(self, in_channels, out_channels, stride=1, expansion=6, downsample=None):
    super(block, self).__init__()
    middle_channels = in_channels*expansion

    self.pass1 = nn.Sequential(
        nn.Conv2d(in_channels, middle_channels, 1, stride=1, padding=0, bias=False), # self conv
        nn.BatchNorm2d(middle_channels),
        # nn.Dropout(0.1),
        nn.ReLU6(),
        nn.Conv2d(middle_channels, middle_channels, 3, stride=stride, padding=1, bias=False), # downsamples if stride=2
        nn.BatchNorm2d(middle_channels),
        # nn.Dropout(0.1),
        nn.ReLU6(),
        nn.Conv2d(middle_channels, out_channels, 1, stride=1, padding=0, bias=False),
        nn.BatchNorm2d(out_channels),
        # nn.Dropout(0.1),
        nn.ReLU6(),
    )

    self.pass2 = nn.Sequential(
        nn.Conv2d(in_channels, in_channels*expansion, kernel_size=3, stride=1, padding=1, bias=False),
        nn.BatchNorm2d(in_channels*expansion),
        # nn.Dropout(0.1),
        nn.ReLU6(),
        nn.Conv2d(in_channels*expansion, in_channels*expansion, kernel_size=3, stride=stride, padding=1,groups=in_channels, bias=False),
        nn.BatchNorm2d(in_channels*expansion),
        # nn.Dropout(0.1),
        nn.ReLU6(),
        nn.Conv2d(in_channels*expansion, out_channels, kernel_size=1, stride=1, bias=False),
        nn.BatchNorm2d(out_channels),
        # nn.Dropout(0.1),
        nn.ReLU6(),
    )

    self.downsample = downsample
  
  def forward(self, x):
    identity = x.clone()
    x1 = self.pass1(x)
    x2 = self.pass2(x)
    x = x1 + x2
    if self.downsample is not None:
      identity = self.downsample(identity)
      x += identity
    return x



class Network(nn.Module):
  def __init__(self, in_channels=3, num_classes=10):
    super(Network, self).__init__()
    self.conv1 = nn.Conv2d(in_channels, 32, kernel_size=7, stride=2, padding=3, bias=False)
    self.bn1 = nn.BatchNorm2d(32)
    self.relu6 = nn.ReLU6()
    self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

    self.block1 = self._make_layer(block, 1, 32, 16, 1, 1)
    self.block2 = self._make_layer(block, 6, 16, 24, 2, 2)
    self.block3 = self._make_layer(block, 6, 24, 32, 3, 2)
    self.block4 = self._make_layer(block, 6, 32, 64, 4, 2)
    self.block5 = self._make_layer(block, 6, 64, 96, 3, 1)
    self.block6 = self._make_layer(block, 6, 96, 160, 3, 2)
    self.block7 = self._make_layer(block, 6, 160, 320, 1, 1)
    
    self.conv2 = nn.Conv2d(320,1280, kernel_size=3, stride=1, padding=1, bias=False)
    self.bn2 = nn.BatchNorm2d(1280)
    self.avgPool = nn.AvgPool2d(7)
    self.flatten = nn.Flatten()
    self.fc = nn.Linear(1280, num_classes)

  def forward(self, x):
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu6(x)
    x = self.block1(x)
    x = self.block2(x)
    x = self.block3(x)
    x = self.block4(x)
    x = self.block5(x)
    x = self.block6(x)
    x = self.block7(x)
    x = self.conv2(x)
    x = self.bn2(x)
    x = self.relu6(x)
    x = self.avgPool(x)
    x = self.flatten(x)
    x = self.fc(x)
    x = F.log_softmax(x, dim=1)
    
    return x
  
  def _make_layer(self, block, expansion, in_channels, out_channels, repeats, stride):
    layers = []
    downsample = None

    if stride != 1:
      downsample = nn.Sequential(
          nn.Conv2d(in_channels,out_channels,kernel_size=1,stride=stride, bias=False),
          nn.BatchNorm2d(out_channels),
      )


    layers.append(
        block(in_channels, out_channels, stride, expansion, downsample)
    )

    for _ in range(repeats-1):
      layers.append(
          block(out_channels, out_channels, 1, expansion)
      )
    
    return nn.Sequential(*layers)


model = Network()

path = '/content/drive/MyDrive/Colab Notebooks/mix2 adam 31-70ep 90-83.pth'
model.load_state_dict(torch.load(path))

<All keys matched successfully>

In [5]:
# ResNet: Not own code (20 epochs, % training, 17% testing)

class block(nn.Module):
    def __init__(
        self, in_channels, intermediate_channels, identity_downsample=None, stride=1
    ):
        super(block, self).__init__()
        self.expansion = 4
        self.conv1 = nn.Conv2d(
              in_channels, intermediate_channels, kernel_size=1, stride=1, padding=0, bias=False
          ) # [_, int_channels, x,y]
        self.bn1 = nn.BatchNorm2d(intermediate_channels)
        self.conv2 = nn.Conv2d(
            intermediate_channels,
            intermediate_channels,
            kernel_size=3,
            stride=stride,
            padding=1,
            bias=False
        ) # [_, int_channels, x,y]
        self.bn2 = nn.BatchNorm2d(intermediate_channels)
        self.conv3 = nn.Conv2d(
            intermediate_channels,
            intermediate_channels * self.expansion,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=False
        ) # [_, int_channels*4, x, y]
        self.bn3 = nn.BatchNorm2d(intermediate_channels * self.expansion)
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample
        self.stride = stride

    def forward(self, x):
        identity = x.clone()

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.bn3(x)

        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)

        x += identity
        x = self.relu(x)
        return x


class ResNet(nn.Module):
    def __init__(self, block, layers, image_channels, num_classes):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(image_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Essentially the entire ResNet architecture are in these 4 lines below
        self.layer1 = self._make_layer(
            block, layers[0], intermediate_channels=64, stride=1
        )
        self.layer2 = self._make_layer(
            block, layers[1], intermediate_channels=128, stride=2
        )
        self.layer3 = self._make_layer(
            block, layers[2], intermediate_channels=256, stride=2
        )
        self.layer4 = self._make_layer(
            block, layers[3], intermediate_channels=512, stride=2
        )

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * 4, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)

        return x

    def _make_layer(self, block, num_residual_blocks, intermediate_channels, stride):
        identity_downsample = None
        layers = []

        # Either if we half the input space for ex, 56x56 -> 28x28 (stride=2), or channels changes
        # we need to adapt the Identity (skip connection) so it will be able to be added
        # to the layer that's ahead
        if stride != 1 or self.in_channels != intermediate_channels * 4:
            identity_downsample = nn.Sequential(
                nn.Conv2d(
                    self.in_channels,
                    intermediate_channels * 4,
                    kernel_size=1,
                    stride=stride,
                    bias=False
                ),
                nn.BatchNorm2d(intermediate_channels * 4),
            )

        layers.append(
            block(self.in_channels, intermediate_channels, identity_downsample, stride)
        )

        # The expansion size is always 4 for ResNet 50,101,152
        self.in_channels = intermediate_channels * 4

        # For example for first resnet layer: 256 will be mapped to 64 as intermediate layer,
        # then finally back to 256. Hence no identity downsample is needed, since stride = 1,
        # and also same amount of channels.
        for i in range(num_residual_blocks - 1):
            layers.append(block(self.in_channels, intermediate_channels))

        return nn.Sequential(*layers)
# model = ResNet(block, [3,4,6,3], image_channels=3, num_classes=10)

Main Training Loop

In [6]:
from torch.autograd import Variable
from torch.optim import Adam, SGD, lr_scheduler

# Function to save the model
def saveModel():
    path = '/content/drive/MyDrive/Colab Notebooks/mix2 curr.pth'
    torch.save(model.state_dict(), path)

# Function to test the model with the test dataset and print the accuracy for the test images
def testAccuracy(device):
    
    model.eval()
    accuracy = 0.0
    total = 0.0
    
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images = Variable(images.to(device))
            labels = Variable(labels.to(device))
            # run the model on the test set to predict labels
            outputs = model(images)
            # the label with the highest energy will be our prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            accuracy += (predicted == labels).sum().item()
    
    # compute the accuracy over all test images
    accuracy = (100 * accuracy / total)
    return(accuracy)

def trainAccuracy(device):

    model.eval()
    accuracy = 0.0
    total = 0.0
    
    with torch.no_grad():
        for data in train_loader:
            images, labels = data
            images = Variable(images.to(device))
            labels = Variable(labels.to(device))
            # run the model on the test set to predict labels
            outputs = model(images)
            # the label with the highest energy will be our prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            accuracy += (predicted == labels).sum().item()
    
    # compute the accuracy over all test images
    accuracy = (100 * accuracy / total)
    return(accuracy)

def train(num_epochs):
    
    best_accuracy = 0.0

    # Define your execution device
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("The model will be running on", device, "device")
    # Convert model parameters and buffers to CPU or Cuda
    model.to(device)
    model.train()
    # Define the loss function with Classification Cross-Entropy loss and an optimizer with Adam optimizer
    loss_fn = nn.CrossEntropyLoss()
    # optimizer = SGD(model.parameters(), lr=0.0005, momentum=0.9, weight_decay=0.00004)
    optimizer = Adam(model.parameters(), lr=0.00024, weight_decay = 0.00001)
    scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=0.98)
    all_accuracy = []

    for epoch in range(num_epochs):  # loop over the dataset multiple times
        running_loss = 0.0
        model.train()
        for i, (images, labels) in enumerate(train_loader, 0):
            # get the inputs
            images = Variable(images.to(device))
            labels = Variable(labels.to(device))

            # zero the parameter gradients
            optimizer.zero_grad()
            # predict classes using images from the training set
            outputs = model(images)
            # compute the loss based on model output and real labels
            loss = loss_fn(outputs, labels)
            # backpropagate the loss
            loss.backward()
            # adjust parameters based on the calculated gradients
            optimizer.step()

            # Let's print statistics for every 50 images
            # running_loss += loss.item()     # extract the loss value
            # if i % 10 == 10:    
            #     # print every 50 (twice per epoch) 
            #     print('[%d, %5d] loss: %.8f' %
            #           (epoch + 1, i + 1, running_loss / 1000))
            #     # zero the loss
            #     running_loss = 0.0

        # Compute and print the average accuracy for this epoch when tested over all test images
        model.eval()
        accuracy = testAccuracy(device)
        train_accuracy = trainAccuracy(device)
        all_accuracy.append(accuracy)
        print('For epoch', epoch+1,'the train accuracy is %d %%' % (train_accuracy), 'the test accuracy over the whole test set is %d %%' % (accuracy))
        
        # we want to save the model if the accuracy is the best
        if accuracy > best_accuracy:
            saveModel()
            best_accuracy = accuracy

        scheduler.step()
    print('The average accuracy over %d' % num_epochs, ' runs is %.2f' % (sum(all_accuracy)/num_epochs))

Showing Images

In [7]:
# Function to show the images
def imageshow(img):
    img = img / 2 + 0.5     # unnormalize
    if torch.cuda.is_available():
      npimg = img.cpu().numpy()
    else:
      npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [8]:
def checkTransformedImages():
    # get batch of images from the test DataLoader
    images, labels = next(iter(test_loader))
    if torch.cuda.is_available():
      images = images.cuda()
      labels = labels.cuda()

    # show all images as one image grid
    imageshow(torchvision.utils.make_grid(images))

Run Block

In [9]:
train_loader, test_loader = load_data()

num_epochs = 40
train(num_epochs)

checkTransformedImages()

The model will be running on cuda:0 device
For epoch 1 the train accuracy is 92 % the test accuracy over the whole test set is 85 %
For epoch 2 the train accuracy is 92 % the test accuracy over the whole test set is 84 %
For epoch 3 the train accuracy is 92 % the test accuracy over the whole test set is 85 %
For epoch 4 the train accuracy is 92 % the test accuracy over the whole test set is 84 %
For epoch 5 the train accuracy is 93 % the test accuracy over the whole test set is 84 %
For epoch 6 the train accuracy is 93 % the test accuracy over the whole test set is 85 %
For epoch 7 the train accuracy is 93 % the test accuracy over the whole test set is 86 %
For epoch 8 the train accuracy is 92 % the test accuracy over the whole test set is 85 %
For epoch 9 the train accuracy is 93 % the test accuracy over the whole test set is 85 %
For epoch 10 the train accuracy is 93 % the test accuracy over the whole test set is 84 %
For epoch 11 the train accuracy is 93 % the test accuracy over the

KeyboardInterrupt: ignored